# Proyecto - Asesor legal para consultar historia de demandas.
## Preparación de datos

Para esta parte del proyecto se seguirá el siguiente esquema en código 

```mermaid
    graph LR 
        A[ 1 <br> Carga y limpieza base]-->B[2 <br> Campos adicionales para filtrado y análisis]
        B-->C[3 <br> Construcción del campo documento para RAG ]
        C-->D[4 <br> Filtrado para esta PoC]
        D-->E[5 <br> Guardar datasets limpios]
```

In [6]:
import pandas as pd
import numpy as np

### 1. Carga y limpieza base

In [7]:
PATH = "C:/Users/nat27/Desktop/Desktop/Proyectos/CienciaDatos/legal-consulting-poc/data/raw/sentencias_pasadas.xlsx"
df = pd.read_excel(PATH)

In [8]:
# Renombrar columnas a snake_case 
df = df.rename(columns={
    '#': 'id_raw',
    'Relevancia': 'relevancia',
    'Providencia': 'providencia',
    'Tipo': 'tipo',
    'Fecha Sentencia': 'fecha_sentencia',
    'Tema - subtema': 'tema_subtema',
    'resuelve': 'resuelve',
    'sintesis': 'sintesis'
})

# Asegurar tipos
df['fecha_sentencia'] = pd.to_datetime(df['fecha_sentencia'], errors='coerce')

# Limpiar texto (quitar espacios extra)
def clean_text(x):
    if pd.isna(x):
        return np.nan
    # Normalización básica
    txt = str(x).strip()
    # Reemplazos suaves de espacios
    txt = " ".join(txt.split())
    return txt

for col in ['providencia', 'tema_subtema', 'resuelve', 'sintesis', 'tipo']:
    df[col] = df[col].apply(clean_text)

# Rellenar faltantes
df['tipo'] = df['tipo'].fillna('Desconocido')
df['tema_subtema'] = df['tema_subtema'].fillna('No especificado')

# Crear un id estable (por si el Excel cambia de orden)
df = df.reset_index(drop=True)
df['doc_id'] = df.index.astype(str)

### 2. Campos adicionales para filtrado y análisis

In [9]:
# Año para filtros rápidos
df['anio'] = df['fecha_sentencia'].dt.year

# Bandera de redes sociales (ajusta/expande el diccionario de keywords según tu conjunto)
social_keywords = [
    'redes sociales','facebook','instagram','twitter','x (twitter)','tiktok','youtube',
    'whatsapp','telegram','mensajería', 'plataforma digital', 'publicación', 'post',
    'comentario', 'perfil', 'cuenta', 'historia', 'story'
]

def contains_any(text, keywords):
    if not isinstance(text, str):
        return False
    t = text.lower()
    return any(k in t for k in keywords)

# Marcamos si el caso parece estar vinculado a redes sociales usando varias columnas
df['is_redes_sociales'] = (
    df['tema_subtema'].apply(lambda x: contains_any(x, social_keywords)) |
    df['sintesis'].apply(lambda x: contains_any(x, social_keywords)) |
    df['resuelve'].apply(lambda x: contains_any(x, social_keywords)) |
    df['providencia'].apply(lambda x: contains_any(x, social_keywords))
)

# Marcadores específicos que te piden en la prueba
df['is_acoso_escolar'] = (
    df['tema_subtema'].str.lower().str.contains('acoso escolar', na=False) |
    df['sintesis'].str.lower().str.contains('acoso escolar', na=False)
)

df['mentions_piar'] = (
    df['tema_subtema'].str.lower().str.contains('piar', na=False) |
    df['sintesis'].str.lower().str.contains('piar', na=False) |
    df['resuelve'].str.lower().str.contains('piar', na=False)
)

### 3. Construcción del campo "documento" para RAG 

In [10]:
def build_document(row):
    partes = []
    partes.append(f"Providencia: {row['providencia']}" if pd.notna(row['providencia']) else "")
    partes.append(f"Tema/Subtema: {row['tema_subtema']}" if pd.notna(row['tema_subtema']) else "")
    partes.append(f"Sintesis: {row['sintesis']}" if pd.notna(row['sintesis']) else "")
    partes.append(f"Resuelve: {row['resuelve']}" if pd.notna(row['resuelve']) else "")
    partes.append(f"Fecha sentencia: {row['fecha_sentencia'].date() if pd.notna(row['fecha_sentencia']) else 'Desconocida'}")
    partes.append(f"Relevancia: {row['relevancia']}" if pd.notna(row['relevancia']) else "")
    partes.append(f"Tipo: {row['tipo']}" if pd.notna(row['tipo']) else "")
    # Unimos con saltos de línea para mejorar la legibilidad en el contexto del LLM
    return "\n".join([p for p in partes if p])

df['documento'] = df.apply(build_document, axis=1)

### 4. Filtrado para esta PoC: solo casos vinculados a redes sociales

In [11]:
df_rag = df[df['is_redes_sociales']].copy()

print(f"Total casos: {len(df)} | Casos vinculados a redes sociales: {len(df_rag)}")

Total casos: 329 | Casos vinculados a redes sociales: 180


### 5. Guardar datasets limpios

In [13]:
PATH_OUTPUT = "C:/Users/nat27/Desktop/Desktop/Proyectos/CienciaDatos/legal-consulting-poc/data/clean/"
df.to_csv(PATH_OUTPUT + "sentencias_limpio_full.csv", index=False, encoding="utf-8")

df_rag[['doc_id','documento','is_acoso_escolar','mentions_piar','anio']].to_csv(
    PATH_OUTPUT + "sentencias_rag_base.csv", index=False, encoding="utf-8"
)
# Vista rápida
df_rag[['doc_id','tema_subtema','is_acoso_escolar','mentions_piar']].head(10)

,doc_id,tema_subtema,is_acoso_escolar,mentions_piar
0,0,No especificado,False,False
1,1,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,False,False
2,2,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,False,False
3,3,ACCION DE TUTELA PARA PROTEGER EL DERECHO A LA...,False,False
4,4,ACCION DE TUTELA-Inexistencia de hecho superad...,False,False
5,5,No especificado,False,False
6,6,No especificado,False,False
7,7,DERECHO A LA INTIMIDAD-Alcance y contenido DER...,False,False
8,8,CARENCIA ACTUAL DE OBJETO POR DAÑO CONSUMADO-D...,False,False
9,9,ACCION DE TUTELA CONTRA PARTICULARES-Solicitud...,False,False
